# FileHashDedupToUSF.ipynb
Notebook for interactive stepwise execution of https://github.com/IGES-Geospatial/MHM_PhotoDeduplicator/blob/main/FileHashDedupToUSF.py to permit dicussion and understanding of processing steps

In [1]:
# Import Libraries
import pandas as pd
import csv

In [2]:
# Set Notebook Display Options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_colwidth", None)

In [3]:
# Load the "Photo First" running record of image filehashes, read from Parquet file on Github
parquet_path = 'https://github.com/IGES-Geospatial/MHM_PhotoDeduplicator/raw/main/MHM_FileHashes.parquet'
df = pd.read_parquet(parquet_path)
display(df.head())
print(df.shape)

,ThumbnailSHA256Hash,ThumbnailUrl,protocol,DataSource,MeasuredAt,PhotoUrl,PhotoType,WaterSourceType,WaterSource,MeasurementLatitude,MeasurementLongitude,LocationMethod,LocationAccuracyM,MosquitoHabitatMapperId,Userid,LastIdentifyStage,Genus,Species,MosquitoAdults,MosquitoPupae,LarvaeCount,MosquitoEggs,MosquitoEggCount,Comments,BreedingGroundEliminated,GlobeTeams,organizationId,organizationName,siteId,siteName,ExtraData,MeasurementElevation
0,7853c6c34833390717d594eba23062cd16c7522dd9d30348289ddbf6d00a48ac,https://data.globe.gov/system/photos/2022/11/17/3153107/thumb.jpg,mosquito_habitat_mapper,GLOBE Observer App,2022-11-17T03:08:00,https://data.globe.gov/system/photos/2022/11/17/3153107/original.jpg,LarvaFullBodyPhoto,still: lake/pond/swamp,pond,14.6356,104.6488,automatic,35,39175,78711887,identify-siphon-hairs,null,null,false,true,33,false,null,null,true,null,2852160,Krachaengwittaya School,297387,48PVB621180,LarvaeVisibleYes,0
1,d8660b6ccb14a7a24db03e03027035d8a435479010358006cd39c685f83e8970,https://data.globe.gov/system/photos/2022/11/17/3153108/thumb.jpg,mosquito_habitat_mapper,GLOBE Observer App,2022-11-17T03:08:00,https://data.globe.gov/system/photos/2022/11/17/3153108/original.jpg,LarvaFullBodyPhoto,still: lake/pond/swamp,pond,14.6356,104.6488,automatic,35,39175,78711887,identify-siphon-hairs,null,null,false,true,33,false,null,null,true,null,2852160,Krachaengwittaya School,297387,48PVB621180,LarvaeVisibleYes,0
2,37174b9ee00e53f875382ba5d3174d37102c3572b726921611842792f87a9f9d,https://data.globe.gov/system/photos/2022/11/17/3153109/thumb.jpg,mosquito_habitat_mapper,GLOBE Observer App,2022-11-17T03:08:00,https://data.globe.gov/system/photos/2022/11/17/3153109/original.jpg,LarvaFullBodyPhoto,still: lake/pond/swamp,pond,14.6356,104.6488,automatic,35,39175,78711887,identify-siphon-hairs,null,null,false,true,33,false,null,null,true,null,2852160,Krachaengwittaya School,297387,48PVB621180,LarvaeVisibleYes,0
3,8f9b1aa82861bc9ca2c3a911775a68c605d945acf0ebf48456654ef08638148f,https://data.globe.gov/system/photos/2022/11/17/3153110/thumb.jpg,mosquito_habitat_mapper,GLOBE Observer App,2022-11-17T03:08:00,https://data.globe.gov/system/photos/2022/11/17/3153110/original.jpg,LarvaFullBodyPhoto,still: lake/pond/swamp,pond,14.6356,104.6488,automatic,35,39175,78711887,identify-siphon-hairs,null,null,false,true,33,false,null,null,true,null,2852160,Krachaengwittaya School,297387,48PVB621180,LarvaeVisibleYes,0
4,310d47f4d6db179b2b39cd1ca181245fcbc972a49c526a1f161044739969dddb,https://data.globe.gov/system/photos/2022/11/10/3143585/thumb.jpg,mosquito_habitat_mapper,GLOBE Observer App,2022-11-10T00:46:00,https://data.globe.gov/system/photos/2022/11/10/3143585/original.jpg,LarvaFullBodyPhoto,container: artificial,"cement, metal or plastic tank",16.9156,-89.8998,automatic,20,39158,47417614,identify-no-siphon,Anopheles,null,true,true,100,true,null,null,true,[TS El Caoba],19879312,Guatemala Citizen Science,296741,16QAD910724,LarvaeVisibleYes,0


(66590, 32)


In [4]:
# Drop all photos (rows) with duplicate filehashes
# Function Documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
df.drop_duplicates(subset=['ThumbnailSHA256Hash'], keep=False, inplace=True, ignore_index=True)
print(df.shape)

(49673, 32)


In [5]:
# Drop "photo first" columns - as they are not being used outside of the running filehash record
df.drop(columns=['ThumbnailSHA256Hash', 'ThumbnailUrl'], inplace=True)

In [6]:
# Recombine photo urls into lists according to photo type - matching format of original GO data
# There is likely a more elegant way to achieve this

# Water Source Photos
## Create a dataframe of all water source photos
w = df[df['PhotoType'].str.contains('WaterSourcePhoto')==True]
w = w.drop(columns=['PhotoType'])
## Combine all water source photos for a given observation into a list held in a single column "WaterSourcePhotoUrls"
w_map = {col: "first" for col in w.columns}
w_map["PhotoUrl"] = list
w = w.groupby(["MosquitoHabitatMapperId"], as_index=False).agg(w_map)
w.rename(columns = {'PhotoUrl':'WaterSourcePhotoUrls'}, inplace = True)
## Reformat from list into a semicolon-delimited string, as is reported by the GLOBE API
w["WaterSourcePhotoUrls"] = w["WaterSourcePhotoUrls"].str.join("; ")

# Larva Full Body Photos
## Create a dataframe of all larva full body photos
l = df[df['PhotoType'].str.contains('LarvaFullBodyPhoto')==True]
l = l.drop(columns=['PhotoType'])
## Combine all larva full body photos for a given observation into a list held in a single column "LarvaFullBodyPhotoUrls"
l_map = {col: "first" for col in l.columns}
l_map["PhotoUrl"] = list
l = l.groupby(["MosquitoHabitatMapperId"], as_index=False).agg(l_map)
l.rename(columns = {'PhotoUrl':'LarvaFullBodyPhotoUrls'}, inplace = True)
## Reformat from list into a semicolon-delimited string, as is reported by the GLOBE API
l["LarvaFullBodyPhotoUrls"] = l["LarvaFullBodyPhotoUrls"].str.join("; ")

# Abdomen Closeup Photos
## Create a dataframe of all abdomen closeup photos
a = df[df['PhotoType'].str.contains('AbdomenCloseupPhoto')==True]
a = a.drop(columns=['PhotoType'])
## Combine all abdomen closeup photos for a given observation into a list held in a single column "LarvaFullBodyPhotoUrls"
a_map = {col: "first" for col in a.columns}
a_map["PhotoUrl"] = list
a = a.groupby(["MosquitoHabitatMapperId"], as_index=False).agg(a_map)
a.rename(columns = {'PhotoUrl':'AbdomenCloseupPhotoUrls'}, inplace = True)
## Reformat from list into a semicolon-delimited string, as is reported by the GLOBE API
a["AbdomenCloseupPhotoUrls"] = a["AbdomenCloseupPhotoUrls"].str.join("; ")

#Concat into one dataframe with rows containing combined photo lists for each type
df = pd.concat([l,w,a], ignore_index=True).groupby(["MosquitoHabitatMapperId"], as_index=False).first()

df.shape


(28478, 31)

In [7]:
# Append Date Columns from JSON API response, matched on observation ID

# Import Libraries
import requests
import json
import datetime

# Define the GLOBE API request URL

base_url = "https://api.globe.gov/search/v1/measurement/protocol/measureddate/"
requestParameters = {
    "protocols": "mosquito_habitat_mapper",
    "startdate": "2017-05-01",
    "enddate" : datetime.datetime.utcnow().date(),
    "geojson" : "FALSE", 
    "sample" : "FALSE"
}


# Make the request
response = requests.get(base_url, params=requestParameters)
# Keep the results
results = response.json()["results"]

# Pass the results as a Dataframe
df_t = pd.DataFrame(results)

# Expand the nested 'data' column by listing the contents and passing as a new dataframe
df_t = pd.concat([df_t, pd.DataFrame(list(df_t['data']))], axis=1)

#Drop the previously nested data column
df_t = df_t.drop('data', axis=1)

#Rename/Shorten Columns for complete display
df_t.columns = df_t.columns.str.replace('mosquitohabitatmapper', '')

# Keep the temporal columns and MosquitoHabitatMapperId
df_t = df_t[['MosquitoHabitatMapperId', 'measuredDate', 'createDate', 'updateDate', 'publishDate']]

# Treat all columns as strings
df_t = df_t.astype(str)

In [8]:
# Append temporal fields, matched on MosquitoHabitatMapperId
df = df.merge(df_t, how='inner', on='MosquitoHabitatMapperId', suffixes=(False, False))

In [9]:
# Match formatting of original fields per USF requirements
# Replace NA values with empty string
df = df.fillna('')
# Replace 'null' strings with empty strings
df = df.replace('null', '', regex=False)
# Add "mhm_" prefix
df = df.add_prefix('mhm_')
# Match USF field names
df.rename(columns = {'mhm_MeasurementLatitude':'mhm_Latitude', 'mhm_MeasurementLongitude':'mhm_Longitude'}, inplace = True)
# Match USF field order
df = df[['mhm_protocol', 'mhm_measuredDate', 'mhm_createDate', 'mhm_updateDate', 'mhm_publishDate', 'mhm_organizationId', 'mhm_organizationName', 'mhm_siteId', 'mhm_siteName', 'mhm_ExtraData', 'mhm_AbdomenCloseupPhotoUrls', 'mhm_LarvaeCount', 'mhm_MosquitoEggs', 'mhm_LocationAccuracyM', 'mhm_MosquitoEggCount', 'mhm_Comments', 'mhm_WaterSourcePhotoUrls', 'mhm_Latitude', 'mhm_Longitude', 'mhm_MosquitoHabitatMapperId', 'mhm_BreedingGroundEliminated', 'mhm_MeasuredAt', 'mhm_MeasurementElevation', 'mhm_Userid', 'mhm_Genus', 'mhm_LocationMethod', 'mhm_WaterSource', 'mhm_MosquitoAdults', 'mhm_Species', 'mhm_MosquitoPupae', 'mhm_DataSource', 'mhm_LarvaFullBodyPhotoUrls', 'mhm_LastIdentifyStage', 'mhm_WaterSourceType', 'mhm_GlobeTeams']]
# Drop any rows missing Latitude or Longitude values
df = df[df.mhm_Latitude != '']
df = df[df.mhm_Longitude != '']

In [10]:
df.shape

(28476, 35)

In [11]:
df

,mhm_protocol,mhm_measuredDate,mhm_createDate,mhm_updateDate,mhm_publishDate,mhm_organizationId,mhm_organizationName,mhm_siteId,mhm_siteName,mhm_ExtraData,mhm_AbdomenCloseupPhotoUrls,mhm_LarvaeCount,mhm_MosquitoEggs,mhm_LocationAccuracyM,mhm_MosquitoEggCount,mhm_Comments,mhm_WaterSourcePhotoUrls,mhm_Latitude,mhm_Longitude,mhm_MosquitoHabitatMapperId,mhm_BreedingGroundEliminated,mhm_MeasuredAt,mhm_MeasurementElevation,mhm_Userid,mhm_Genus,mhm_LocationMethod,mhm_WaterSource,mhm_MosquitoAdults,mhm_Species,mhm_MosquitoPupae,mhm_DataSource,mhm_LarvaFullBodyPhotoUrls,mhm_LastIdentifyStage,mhm_WaterSourceType,mhm_GlobeTeams
0,mosquito_habitat_mapper,2017-05-31,2022-02-22T08:37:26,2022-02-22T08:37:26,2022-06-28T16:50:17,14564129,"SciStarter Citizen Scientists, LLC GLOBE v-School",47350,16SEE200984,,,,,,,,https://data.globe.gov/system/photos/2017/05/31/167552/original.jpg,36.130087,-86.777727,10,true,2017-05-31T20:10:00,165.8,21767934,,,other,,,,GLOBE Observer App,,,container: artificial,
1,mosquito_habitat_mapper,2017-07-06,2022-02-22T08:38:29,2022-02-22T08:38:29,2022-06-28T16:50:17,24610401,Terre Rouge SSS,53208,40KEC556754,,https://data.globe.gov/system/photos/2017/07/06/179670/original.jpg,1-25,false,,,,,-20.117747,57.531913,100,true,2017-07-06T11:19:00,42.9,24612101,,,ditch,true,,true,GLOBE Observer App,https://data.globe.gov/system/photos/2017/07/06/179669/original.jpg,identify-siphon-shape,still: lake/pond/swamp,
2,mosquito_habitat_mapper,2017-09-22,2022-02-22T08:43:56,2022-02-22T08:43:56,2022-06-28T16:50:17,24476638,Escola Estadual Faria Sobrinho,56772,22JGS508753,,,,,,,,https://data.globe.gov/system/photos/2017/09/22/485533/original.jpg; https://data.globe.gov/system/photos/2017/09/22/485534/original.jpg; https://data.globe.gov/system/photos/2017/09/22/485535/original.jpg,-25.518352,-48.504536,1000,false,2017-09-22T19:42:00,12,24476444,,,tree holes,,,,GLOBE Observer App,,,container: natural,
3,mosquito_habitat_mapper,2019-04-02,2022-02-22T09:38:35,2022-02-22T09:38:35,2022-06-28T16:50:17,54203550,CHSE,149726,43NCE338612,,,,,,,,https://data.globe.gov/system/photos/2019/04/02/1017686/original.jpg,4.1715,73.5033,10000,false,2019-04-02T09:21:00,0,54205784,,,"cement, metal or plastic tank",,,false,GLOBE Observer App,,identify,container: artificial,
4,mosquito_habitat_mapper,2019-04-02,2022-02-22T09:38:35,2022-02-22T09:38:35,2022-06-28T16:50:17,14054356,lycee Thilmakha,148574,28PDB042456,,,20,true,,,,https://data.globe.gov/system/photos/2019/04/02/1017711/original.jpg,14.8833,-15.8897,10004,true,2019-04-02T20:50:00,0,51045191,,,"cement, metal or plastic tank",false,,true,GLOBE Observer App,,identify-siphon-shape,container: artificial,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28473,mosquito_habitat_mapper,2017-09-19,2022-02-22T08:43:56,2022-02-22T08:43:56,2022-06-28T16:50:17,24476638,Escola Estadual Faria Sobrinho,56773,22JGS508754,,,,,,,Difícil acesso para a amostra e coleta...,https://data.globe.gov/system/photos/2017/09/19/485530/original.jpg; https://data.globe.gov/system/photos/2017/09/19/485531/original.jpg; https://data.globe.gov/system/photos/2017/09/19/485532/original.jpg,-25.51745,-48.504555,999,false,2017-09-19T19:32:00,12.5,24476444,,,well or cistern,,,,GLOBE Observer App,,,container: artificial,
28474,mosquito_habitat_mapper,2019-04-02,2022-02-22T09:38:35,2022-02-22T09:38:35,2022-06-28T16:50:17,14054356,lycee Thilmakha,143313,28PCB659632,,,13,true,,,,https://data.globe.gov/system/photos/2019/04/02/1017500/original.jpg,15.041,-16.2473,9990,true,2019-04-02T04:48:00,0,51045191,,,"cement, metal or plastic tank",false,,true,GLOBE Observer App,,identify-siphon-shape,container: artificial,
28475,mosquito_habitat_mapper,2019-04-02,2022-02-22T09:38:35,2022-02-22T09:38:35,2022-06-28T16:50:17,14054356,lycee Thilmakha,143313,28PCB659632,,,10,true,,,,https://data.globe.gov/system/photos/2019/04/02/1017516/original.jpg; https://data.g